In [80]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
import scipy.stats as stats
import numpy as np

In [81]:
#Section 1, Data generation, define NN model
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_layers, output_size, std_deviation):
        super(SimpleNN, self).__init__()
        self.hidden_layers = nn.ModuleList()
        previous_size = input_size
        self.std_dev = abs(std_deviation)
        for layer_size in hidden_layers:
            self.hidden_layers.append(nn.Linear(previous_size, layer_size))
            previous_size = layer_size
        self.output_layer = nn.Linear(previous_size, output_size)


    def forward(self, x, add_noise = True):
        for layer in self.hidden_layers:
            x = torch.relu(layer(x))
        if add_noise == True: # add noise to each output
            noise = torch.randn_like(x) * self.std_dev
        return self.output_layer(x) + noise


input_size, hidden_layers, output_size = 3, [3, 3], 1  # L = 2 K = 3, is the "output_size = 1" an architextrue pram of interest?
std_dev = 1.3 #true standard deviation of noise

model = SimpleNN(input_size, hidden_layers, output_size, std_dev)

# Manually set the weights and biases for each neuron
#layer 1
model.hidden_layers[0].weight.data.fill_(0.12)
model.hidden_layers[0].bias.data.fill_(0.2)
#layer 2
model.hidden_layers[1].weight.data.fill_(0.11)
model.hidden_layers[1].bias.data.fill_(0.12)
#output layer
model.output_layer.weight.data.fill_(0.13)
model.output_layer.bias.data.fill_(0.001)

#Generate 'X' data
n_samples = 10000 #temp
X = torch.randn(n_samples, input_size)  #normal distributed

#Pass 'X' through the model to generate 'y_true' as outputs
with torch.no_grad():
    y_true = model(X)

#sanity check point 1
print(X, y_true) #How can I perform check without a prediction task ?
print(f"X dimsension: {X.shape}, y_true.shape: {y_true.shape}")
# split data into Da and Db
n_b = n_samples // 2
n_a = n_samples - n_b
Db = TensorDataset(X[:n_b], y_true[:n_b])
Da = TensorDataset(X[n_b:], y_true[n_b:])


train_loader = DataLoader(Db, batch_size=100, shuffle=True)
val_loader = DataLoader(Da, batch_size=100, shuffle=False)

tensor([[-0.0672,  0.0750, -0.0258],
        [-1.3002,  0.5428, -0.1180],
        [ 2.1826,  1.2858, -1.7478],
        ...,
        [-2.3269, -0.1246, -0.1468],
        [-0.1641, -1.5404, -1.2181],
        [ 0.2371, -0.9498,  1.8402]]) tensor([[-1.6092, -2.9530,  3.1714],
        [ 2.1529,  2.1953, -1.1351],
        [ 0.5223,  1.0914,  0.3954],
        ...,
        [-0.0418,  0.2147, -1.1560],
        [ 1.0544,  0.2339,  1.0190],
        [ 0.6998,  0.9911, -0.2495]])
X dimsension: torch.Size([10000, 3]), y_true.shape: torch.Size([10000, 3])


In [82]:
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_layers, output_size):
        super(SimpleNN, self).__init__()
        self.hidden_layers = nn.ModuleList()
        previous_size = input_size
        for layer_size in hidden_layers:
            self.hidden_layers.append(nn.Linear(previous_size, layer_size))
            previous_size = layer_size
        self.output_layer = nn.Linear(previous_size, output_size)

    def forward(self, x):
        for layer in self.hidden_layers:
            x = torch.relu(layer(x))
        return self.output_layer(x)

In [83]:
# # section 2.1, Training, Grid Search

# # NN for training with simple MSELoss #How do we know the original model
# class SearchNN(SimpleNN):
#     def __init__(self, input_size, hidden_layers, output_size, lr=0.001):
#         super(SearchNN, self).__init__(input_size, hidden_layers, output_size)
#         self.loss_function = nn.MSELoss() #temperary: MSE
#         self.optimizer = torch.optim.Adam(self.parameters(), lr=lr)

#     def train_single_epoch(self, data_loader):
#         self.train()
#         for x_batch, y_batch in data_loader:
#             self.optimizer.zero_grad()
#             y_pred = self(x_batch)
#             loss = self.loss_function(y_pred, y_batch)
#             loss.backward()
#             self.optimizer.step()
#         return loss.item()

#     def validate(self, data_loader):
#         self.eval()
#         val_losses = []
#         y_test_pred = []
#         residuals = []
#         with torch.no_grad():
#             for x_val, y_val in data_loader:
#                 y_val_pred = self(x_val)
#                 val_loss = self.loss_function(y_val_pred, y_val)
#                 val_losses.append(val_loss.item())

#                 y_val_pred_np = y_val_pred.detach().cpu().numpy()
#                 y_val_np = y_val.cpu().numpy()

#                 y_test_pred.extend(y_val_pred_np)
#                 residuals.extend(y_val_np - y_val_pred_np)


#         return sum(val_losses) / len(val_losses)

# # Function: grid search, training and evaluation
# def train_and_evaluate_nn(grid_search_params, train_loader, val_loader, num_epochs):
#     results = []
#     L, K = grid_search_params[0], grid_search_params[1]
#     #Model with L layers and K units
#     model = SearchNN(input_size, [K] * L, output_size)

#     for epoch in range(num_epochs):
#         train_loss = model.train_single_epoch(train_loader)
#     val_loss = model.validate(val_loader)

#     #results
#     results.append({
#         'L': L,
#         'K': K,
#         'train_loss': train_loss,
#         'val_loss': val_loss
#     })
#     return results

# #Grid search
# grid_search_params = [(l, k) for l in range(1, 4) for k in range(3, 6)] # Small 3x3 grid search
# num_epochs = 5  # temperary
# evaluation_results = {}
# for (l,k) in grid_search_params:
#    evaluation_results[(l,k)] = train_and_evaluate_nn((l,k), train_loader, val_loader, num_epochs)

# print(evaluation_results.items())
# for (L,K), result_list in evaluation_results.items():
#   for result in result_list:
#         train_loss = result['train_loss']
#         val_loss = result['val_loss']
#         print(f"L: {L}, K: {K}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")


In [98]:
# section 2.2, Training, Grid Search with residual and standard deviation packed
class SearchNN(SimpleNN):
    def __init__(self, input_size, hidden_layers, output_size, lr=0.001):
        super(SearchNN, self).__init__(input_size, hidden_layers, output_size)
        self.loss_function = nn.MSELoss() #temperary: MSE
        self.optimizer = torch.optim.Adam(self.parameters(), lr=lr)

    def train_single_epoch(self, data_loader):
        self.train()
        for x_batch, y_batch in data_loader:
            self.optimizer.zero_grad()
            y_pred = self(x_batch)
            loss = self.loss_function(y_pred, y_batch)
            loss.backward()
            self.optimizer.step()
        return loss.item()

    def validate(self, data_loader):
      self.eval()  # Set the model to evaluation mode
      val_losses = []
      all_residuals = []  # Use a list to collect all residuals

      with torch.no_grad():  # No gradients needed for validation
          for x_val, y_val in data_loader:
              y_val_pred = self(x_val)
              val_loss = self.loss_function(y_val_pred, y_val)
              val_losses.append(val_loss.item())

              # Calculate residuals and append them to the list
              residuals = y_val - y_val_pred
              all_residuals.append(residuals)

      # Concatenate all residuals tensors to form a single tensor
      all_residuals_tensor = torch.cat(all_residuals, dim=0)

      # Calculate the standard deviation of residuals
      # This standard deviation is a measure of how spread out these errors are.
      standard_deviation = torch.sqrt(torch.mean(all_residuals_tensor ** 2))

      # The mean validation loss
      mean_val_loss = sum(val_losses) / len(val_losses)

      # Save the residuals and standard deviation as attributes for later use
      self.residuals = all_residuals_tensor
      self.standard_deviation = standard_deviation

      return mean_val_loss


    # def validate(self, data_loader):
    #     self.eval()
    #     val_losses = []
    #     y_val_pred_np = np.array([])
    #     self.residuals = torch.Tensor()
    #     with torch.no_grad():
    #         for x_val, y_val in data_loader:
    #             y_val_pred = self(x_val)
    #             val_loss = self.loss_function(y_val_pred, y_val)
    #             val_losses.append(val_loss.item())

    #             y_val_pred_np = y_val_pred.detach().cpu().numpy()
    #             y_val_np = y_val.cpu().numpy()

    #             self.residuals = torch.cat((self.residuals,  torch.tensor(y_val_np - y_val_pred_np)), 0)
    #     self.standard_deviation = torch.sqrt(torch.sum(torch.pow(self.residuals, 2)) / (self.residuals.numel() - 1))
    #     return sum(val_losses) / len(val_losses)

class Repro():
  def train_and_evaluate_nn(self, grid_search_param, train_loader, val_loader, num_epochs=10):
      results = []
      L, K = grid_search_param
      # Model with L layers and K units
      model = SearchNN(input_size, [K] * L, output_size)

      for epoch in range(num_epochs):
          train_loss = model.train_single_epoch(train_loader)
          #accuracy =
      val_loss = model.validate(val_loader)
      residual = model.residuals
      standard_dev = model.standard_deviation
      size = model.residuals.numel()

      # Define the nuclear mapping function given (L,K) on Da:
      # l(L,K) = -n * log(sqrt(2*pi)*std_dev) - sum(residual**2) / 2*std_dev**2
      likelihood = -size * np.log(np.sqrt(2*np.pi)*standard_dev) - np.sum(torch.pow(residual,2).tolist())/ 2*std_dev**2


      # results
      result = {
          'L': L,
          'K': K,
          'train_loss': train_loss,
          'val_loss': val_loss,
          'residual': residual,
          'standard_dev': standard_dev,
          'log_likelihood': likelihood

      }
      return result

  # def repro_data(self,data,repro_quantity, est_std_dev, seed = 39):
  #   #seed for torch with/o gpu senario
  #   torch.manual_seed(seed)
  #   torch.cuda.manual_seed_all(seed)
  #   # is it the same for calling a random number generator 100 time as calling random generator once for generating 100 random number?
  #   # Yes.
  #   self.noise_std = est_std_dev
  #   self.original_data = data
  #   self.noise = {}
  #   self.newdata_cache = {}

  #   for repro_index in range(repro_quantity):
  #     self.noise[repro_index] = self.noise_std * torch.randn_like(Da.tensors[1])
  #     self.newdata_cashe[repro_index] = self.noise[repro_index] + data
  #   return self.newdata_cache

  def repro_data(self, data, repro_quantity, est_std_dev, seed=39):
    # Set seed for reproducibility
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    self.noise_std = est_std_dev
    self.original_data = data
    self.noise = {}
    self.newdata_cache = {}

    # Extracting tensors from the original TensorDataset
    original_data_tensors = data.tensors

    for repro_index in range(repro_quantity):
        # Generating noise for each tensor in the dataset
        noise_tensors = tuple(self.noise_std * torch.randn_like(tensor) for tensor in original_data_tensors)
        self.noise[repro_index] = noise_tensors

        # Adding noise to the original data tensors
        synthetic_data_tensors = tuple(tensor + noise for tensor, noise in zip(original_data_tensors, noise_tensors))
        self.newdata_cache[repro_index] = TensorDataset(*synthetic_data_tensors)

    return self.newdata_cache


  def monte_carlo(self,data,n_repro,est_std_dev,l_low=1,l_up=4,k_low=3,k_up=6,epoch=5):
    self.grid_search_params = [(l,k) for l in range(l_low, l_up) for k in range(k_low,k_up)]
    self.n_epoch = epoch
    self.n_repro = n_repro
    self.repro_results = {}
    for param in self.grid_search_params:
      data_cache = self.repro_data(data,n_repro,est_std_dev)
      for repro_index, synthetic_data in data_cache.items():
        repro_train_loader = DataLoader(synthetic_data, batch_size=100, shuffle=True)
        self.repro_results[(param,repro_index)] = (repro_index, self.train_and_evaluate_nn(param, repro_train_loader, val_loader, self.n_epoch))

    for (l, k), repro_result_tuple in self.repro_results.items():
        repro_index, result = repro_result_tuple
        train_loss = result['train_loss']
        val_loss = result['val_loss']
        residual = result['residual']
        standard_dev = result['standard_dev']
        likelihood = result['log_likelihood']

        print(f"Architecture:{l}, Repro Index:{repro_index}, Train Loss: {train_loss}, Val Loss: {val_loss}, Std Dev: {standard_dev}, Log Likelihood: {likelihood}")

    #   #check single element tensor and convert to float
    # if isinstance(residual, torch.Tensor) and residual.numel() == 1:
    #     residual = residual.item()
    # if isinstance(standard_dev, torch.Tensor) and standard_dev.numel() == 1:
    #     standard_dev = standard_dev.item()
      # print(f"L:{l}, K:{k}, repro index:{repro_index}, Std:{standard_dev:.4f}, log_likelihood:{likelihood:.4f}")
    # for futher ulization
    return self.repro_results

  def borel_confidence_interval(self, data, alpha = 0.95): # is borel set a confidence interval or a set?
    self.data = np.array(data)
    self.mean = np.mean(data)
    self.n = len(data)
    self.std_dev = np.std(data, ddof=1)
    self.z_score = stats.norm.ppf(1 - (1-alpha)/2) # for over 30, distribution is approx Normal
    self.margin_of_error = self.z_score * (std_dev / np.sqrt(self.n))
    self.confidence_interval = (self.mean - self.margin_of_error, self.mean + self.margin_of_error)
    return self.confidence_interval

# val_loader require initialization


In [99]:
myrepro = Repro()
est_std_dev = Db[:][1].std().item()
n_repro = 5 #should be more than 30, ideally set to 1000
print(est_std_dev)
repro_result = myrepro.monte_carlo(Db,n_repro,est_std_dev)

1.305342674255371


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([100, 3])) that is different to the input size (torch.Size([100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Architecture:(1, 3), Repro Index:0, Train Loss: 4.0392680168151855, Val Loss: 1.7020599842071533, Std Dev: 1.304630160331726, Log Likelihood: -39346.484375
Architecture:(1, 3), Repro Index:1, Train Loss: 3.473264455795288, Val Loss: 1.701017851829529, Std Dev: 1.304230809211731, Log Likelihood: -39328.6796875
Architecture:(1, 3), Repro Index:2, Train Loss: 3.5579569339752197, Val Loss: 1.7073483896255492, Std Dev: 1.3066554069519043, Log Likelihood: -39436.78125
Architecture:(1, 3), Repro Index:3, Train Loss: 3.355862855911255, Val Loss: 1.7019293689727784, Std Dev: 1.3045802116394043, Log Likelihood: -39344.25390625
Architecture:(1, 3), Repro Index:4, Train Loss: 3.5742745399475098, Val Loss: 1.708633198738098, Std Dev: 1.3071469068527222, Log Likelihood: -39458.703125
Architecture:(1, 4), Repro Index:0, Train Loss: 3.8577840328216553, Val Loss: 1.7084450769424437, Std Dev: 1.307075023651123, Log Likelihood: -39455.49609375
Architecture:(1, 4), Repro Index:1, Train Loss: 3.07630348205

In [88]:
# def borel_confidence_interval(self, data, alpha = 0.95): # is borel set a confidence interval or a set?
#     self.data = np.array(data)
#     self.mean = np.mean(data)
#     self.n = len(data)
#     self.std_dev = np.std(data, ddof=1)
#     self.z_score = stats.norm.ppf(1 - (1-alpha)/2) # for over 30, distribution is approx Normal
#     self.margin_of_error = self.z_score * (std_dev / np.sqrt(self.n))
#     self.confidence_interval = (self.mean - self.margin_of_error, mean + self.margin_of_error)
#     return self.confidence_interval

In [100]:
# Empirical Distribution of Likelihood
likelihood_distribution = {}
lt = []
for (l, k), repro_result_tuple in repro_result.items():
    # temp = []
    repro_index, result = repro_result_tuple
    lt.append(result['log_likelihood'].numpy())
    if repro_index == n_repro-1:
        likelihood_distribution[l] = tuple(lt)
        lt.clear()

In [101]:
borel_intervals = {}
for archtect, likelihoods in likelihood_distribution.items():
    data = list(likelihoods)
    interval = myrepro.borel_confidence_interval(data)
    borel_intervals[archtect] = interval

print(borel_intervals)

{(1, 3): (-39384.11994805275, -39381.84098944725), (1, 4): (-39679.08869805275, -39676.80973944725), (1, 5): (-39573.87385430275, -39571.59489569725), (2, 3): (-40131.92854180275, -40129.64958319725), (2, 4): (-39510.89729180275, -39508.61833319725), (2, 5): (-39359.10432305275, -39356.82536444725), (3, 3): (-39642.60041680275, -39640.32145819725), (3, 4): (-39303.99494805275, -39301.71598944725), (3, 5): (-39327.99494805275, -39325.71598944725)}


In [ ]:

#


#Stand alone version
# def train_and_evaluate_nn(grid_search_param, train_loader, val_loader, num_epochs=10):
#     results = []
#     L, K = grid_search_param
#     # Model with L layers and K units
#     model = SearchNN(input_size, [K] * L, output_size)

#     for epoch in range(num_epochs):
#         train_loss = model.train_single_epoch(train_loader)
#         #accuracy =
#     val_loss = model.validate(val_loader)
#     residual = model.residuals
#     standard_dev = model.standard_deviation
#     size = model.residuals.numel()

#     # Define the nuclear mapping function given (L,K) on Da:
#     # l(L,K) = -n * log(sqrt(2*pi)*std_dev) - sum(residual**2) / 2*std_dev**2
#     likelihood = -size * np.log(np.sqrt(2*np.pi)*standard_dev) - np.sum(torch.pow(residual,2).tolist())/ 2*std_dev**2


#     # results
#     result = {
#         'L': L,
#         'K': K,
#         'train_loss': train_loss,
#         'val_loss': val_loss,
#         'residual': residual,
#         'standard_dev': standard_dev,
#         'log_likelihood': likelihood

#     }
#     return result


# # Grid search
# grid_search_params = [(l, k) for l in range(1, 4) for k in range(3, 6)] # Small 3x3 grid search
# num_epochs = 5  # Temporary
# evaluation_results = {}
# for param in grid_search_params:
#     evaluation_results[param] = train_and_evaluate_nn(param, train_loader, val_loader, num_epochs)

# for (L, K), result in evaluation_results.items():
#     train_loss = result['train_loss']
#     val_loss = result['val_loss']
#     residual = result['residual']
#     standard_dev = result['standard_dev']
#     likelihood = result['log_likelihood']

#     #check single element tensor and convert to float
#     if isinstance(residual, torch.Tensor) and residual.numel() == 1:
#         residual = residual.item()
#     if isinstance(standard_dev, torch.Tensor) and standard_dev.numel() == 1:
#         standard_dev = standard_dev.item()

#     print(f"L: {L}, K: {K}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, Residuals: {residual}, Std: {standard_dev:.4f}, log_likelihood: {likelihood: .4f}")

In [ ]:
# # # Repro with nosie (Monte-calo)
# def noiseGenerate(self, size, seed, mean = 0, cov=[[],[]], distribution = 'normal'): # cov need be adjusted to the layer dimensions for each layers
#   self.seed = seed
#   self.mean = mean
#   self.size = size
#   self.cov = cov

# # # Use Quasi Monte-Calo
#   dist = stats.qmc.MultivariateNormalQMC(self.mean, seed=self.seed, cov=self.cov)
#   noise = dist.random(size)
